In [1]:
import pandas as pd
# 용량이 크니 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pwd

'/content'

In [3]:
df = pd.read_csv("./drive/MyDrive/BDA(ML)/수)BDA_추천알고리즘/과제/dataset/2019-Dec.csv")

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3533286 entries, 0 to 3533285
Data columns (total 9 columns):
 #   Column         Dtype  
---  ------         -----  
 0   event_time     object 
 1   event_type     object 
 2   product_id     int64  
 3   category_id    int64  
 4   category_code  object 
 5   brand          object 
 6   price          float64
 7   user_id        int64  
 8   user_session   object 
dtypes: float64(1), int64(3), object(5)
memory usage: 242.6+ MB


In [5]:
# na 여부
df.isna().sum()

,0
event_time,0
event_type,0
product_id,0
category_id,0
category_code,3474821
brand,1510289
price,0
user_id,0
user_session,779


In [6]:
## 데이터 샘플로 뽑아서 분석 진행 _ 5만개, 랜덤씨드는 111
df_sample =df.sample(n=50000, random_state=111)
print(df_sample)

                      event_time        event_type  product_id  \
1141402  2019-12-09 19:04:29 UTC              view     5814516   
2826120  2019-12-23 12:44:51 UTC              view     5915004   
354426   2019-12-03 16:25:54 UTC              view     5877456   
3089023  2019-12-25 19:06:15 UTC              cart     5608703   
1762400  2019-12-14 11:41:25 UTC              view     5769914   
...                          ...               ...         ...   
954138   2019-12-08 14:29:44 UTC  remove_from_cart     5695961   
1363136  2019-12-11 10:17:56 UTC              cart     5800038   
3033914  2019-12-25 10:03:55 UTC          purchase     5629984   
3455329  2019-12-30 08:41:10 UTC          purchase     5851607   
2220662  2019-12-17 21:18:58 UTC              cart     5678044   

                 category_id category_code     brand   price    user_id  \
1141402  1487580013539623112           NaN       NaN   13.49  247790217   
2826120  1487580008028308447           NaN  babyliss   96

## 단순한 추천 로직 (1)
- best5~10은 저번 시간에 이미 코드로 증명
- event_type -> 명시적이진 않고 암묵적인 형태의 데이터인데 -> 그래도 view,  < (cart, purchase) 구매에 대한 확률이 높은 행동
- 가중치를 통한 추천리스트 만들 수 있다.
- e.g. event { view :1점, cart :3점, purchase : 5점 가중치 }
    - product_id 기준으로 가중치 벡터가 만들어진다.
        - 단순한 가중치에 의한 추천을 호출할 수 있다.
        - 선형결합형태로 벡터랑 가중치랑 결합해서 추천스코어가 나올 수 있다.
- 이 형태로 스코어를 계산해 보자!

In [7]:
weights = {
    'view':1,
    'cart':3,
    'purchase':5
}


df_sample['score']=df_sample['event_type'].map(weights)
product_scores = df_sample.groupby('product_id')['score'].sum().reset_index()

## 추천지수가 높은 제품 상위 10개 출력
product_scores=product_scores.sort_values(by='score', ascending=False)
print(product_scores)

       product_id  score
7444      5809910  733.0
3126      5700037  318.0
6785      5802432  300.0
7446      5809912  231.0
11606     5854897  227.0
...           ...    ...
2511      5685400    0.0
15256     5895499    0.0
15252     5895492    0.0
15239     5894931    0.0
17289     5916983    0.0

[17291 rows x 2 columns]


## 단순추천로직 (2)
- KNN 거리반으로 단순하게 추천 가능하다.
    - 추천에 필요한 메트릭스를 생각하면 행과 열에 따라 다르게 나타날 수 있다.
    - 행이 제품이고, 열이 제품일 수 있고 nxn 메트릭스
    - 행이 제품이고, 열이 유저
    - 행이 유정이고, 열이 제품일 수 있다.
    - product_id기준으로 pivot_table 통해 메트릭스 만들기!

In [8]:
### 구매를 한 경우에만 일단 메트릭스를 만들려고 한다.
### 원래는 purchase로 하셔야 하는데 -> 데이터 좀 부족해서 이번만 cart
df_cart =df_sample[df_sample['event_type']=='cart'].copy()

In [9]:
#주문한 데이터만 가지고 만들어 보자!
product_user_matrix =df_cart.pivot_table(index='product_id',columns = 'user_id', aggfunc='size', fill_value=0)

In [10]:
product_user_matrix

user_id,31156111,33679170,40725049,43695999,50142794,56605619,58556296,70406741,72140588,72655920,...,595285565,595286349,595303928,595309103,595315666,595323204,595333128,595343407,595385847,595391039
product_id,,,,,,,,,,,,,,,,,,,,,
3762,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3774,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3928,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3936,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3978,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5916129,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5916410,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5916489,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
## KNN 알고리즘 사용
from sklearn.neighbors import NearestNeighbors

## Cosine 유사도
#product_user_matrix

knn=NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(product_user_matrix)

## KNN 기반 추천 시스템이 구현이 되어야 한다.
## 추천제품을  출력하는 코드 -> input에 따라 다를 것
## A라는 제품을 구매했을 때 -> 같이 구매하면 좋을 장바구니에 담았을 때 같이 추천할 제품을 찾기

def knn_recommendation(selected_proudct, product_user_matrix, knn_model, n_neighbors = 5):
    ## 제품이 추천이 되지 않는 경우 예외조건 설정

    if selected_product not in product_user_matrix.index:
        print(f'제품이 {selected_product}에 대한 데이터가 없습니다.')
        return []

    ## KNN 계산을 하기 위해 선택된 벡터가 필요하고 , knn으로 계산을 진행해야 한다.
    product_vector = product_user_matrix.loc[selected_product].values.reshape(1,-1) # 행,열 맞춤
    distances, indices =knn_model.kneighbors(product_vector, n_neighbors=n_neighbors+1)

    recommendation = []
    # 코사인유사도는 가장 높은 값은 자기자신일 것 그것은 제외하는 조건으로 기준을 나누기도 한다.
    for i in range(1, len(indices[0])):
        recommend_product = product_user_matrix.index[indices[0][i]]
        recommendation.append((recommend_product, 1-distances[0][i]))

    return sorted(recommendation, key = lambda x:x[1], reverse =True)


selected_product = 5916555
recommendation_products =knn_recommendation(selected_product,product_user_matrix,knn )

In [12]:
print(f'제품 {selected_product}과 함께 장바구니에 담을 추천 제품:')
for prod, similarity in recommendation_products:
    print(f'제품 ID{prod}, 유사도: {similarity:.4f}')

제품 5916555과 함께 장바구니에 담을 추천 제품:
제품 ID5916555, 유사도: 1.0000
제품 ID5906929, 유사도: 1.0000
제품 ID5916129, 유사도: 0.0000
제품 ID5916121, 유사도: 0.0000
제품 ID5916018, 유사도: 0.0000


In [13]:
recommendation_products

[(np.int64(5916555), np.float64(1.0)),
 (np.int64(5906929), np.float64(1.0)),
 (np.int64(5916129), np.float64(0.0)),
 (np.int64(5916121), np.float64(0.0)),
 (np.int64(5916018), np.float64(0.0))]

## 단순추천로직(3)
- 동시구매 형태를 살펴보자!
- 동시에 함께 담는 제품들
- 5814516
- 5877456
- (5814516,5877456) 같이 구매한 경우
- 동시에 함께 들어간다 -> 확률, P(A) , P(B) , P(AnB)
- 베이지안으로 생각을 해서 -> 베이지안 확률 모델로 랭킹을 스코어링할 수 있다.

In [14]:
df_sample

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session,score
1141402,2019-12-09 19:04:29 UTC,view,5814516,1487580013539623112,NaN,NaN,13.49,247790217,a897a9a9-5ebe-46df-91a0-db71d9fda2f0,1.0
2826120,2019-12-23 12:44:51 UTC,view,5915004,1487580008028308447,NaN,babyliss,96.51,514193964,5974f0cb-0916-469c-bf62-8843c7208959,1.0
354426,2019-12-03 16:25:54 UTC,view,5877456,1487580006300255120,NaN,jessnail,122.22,580819864,f508ad9a-198e-43f7-9772-e659133f4025,1.0
3089023,2019-12-25 19:06:15 UTC,cart,5608703,1487580005553668971,NaN,NaN,9.52,430430758,d9c35cd6-ce2b-445c-b467-4050a854f1c6,3.0
1762400,2019-12-14 11:41:25 UTC,view,5769914,1487580013841613016,NaN,kapous,3.65,586788406,6248f4cb-e919-4251-afb7-3cac961d787c,1.0
...,...,...,...,...,...,...,...,...,...,...
954138,2019-12-08 14:29:44 UTC,remove_from_cart,5695961,1487580007575323592,NaN,NaN,1.43,448126743,937f8855-e137-4192-9c3f-787bf692317a,NaN
1363136,2019-12-11 10:17:56 UTC,cart,5800038,1487580005385896804,NaN,NaN,5.56,585244747,cb2505b0-7b37-4f15-b9a3-18cb4cd2999c,3.0
3033914,2019-12-25 10:03:55 UTC,purchase,5629984,1487580005754995573,NaN,NaN,4.75,592260899,f6d34c77-c172-4714-be9e-873efe737346,5.0
3455329,2019-12-30 08:41:10 UTC,purchase,5851607,2055161088059638328,NaN,irisk,1.75,563932069,5470ca45-c088-446b-8b6d-dfe7f68e81d4,5.0
